In [1]:
import pandas as pd
data = pd.read_csv('../../cacmData/cacmDataPreprocessed.csv')
data.head()

,.I,data,.B,.C,.N,.X
0,1,preliminari report intern algebra languag perl...,1958-12-01 00:00:00,NaN,1978-03-22 20:28:00,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
1,2,extract root repeat subtract digit comput sugai,1958-12-01 00:00:00,NaN,1978-03-22 20:29:00,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
2,3,techniqu depart matrix program scheme friedman,1958-12-01 00:00:00,NaN,1978-03-22 20:30:00,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
3,4,glossari comput engin program terminolog,1958-11-01 00:00:00,NaN,1978-03-22 20:32:00,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
4,5,two squar root approxim wadey,1958-11-01 00:00:00,NaN,1978-03-22 20:33:00,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

In [3]:
tfidfTable = tfidf.fit_transform(data['data'])

In [4]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

test ='articl exist deal tss time share system oper system ibm comput richard alexand comp serv langmuir lab tss'
# test = 'interest article write eithe prieve pooch prieve pooch author pooch prieve'
# test = 'intermed language use construct multi target compile tcoll donna bergmark comp serv uris hall intermed lang'
# test = '''
# algorithm parallel comput especi comparison parallel sequenti algorithm mark w krentel
# '''
def search(query,n:int):
    querytfidf = tfidf.transform([query])
    cos = linear_kernel(tfidfTable, querytfidf) 
    resultList = cos.argsort(axis=0)[-n:][::-1]
    ls = []
    ids = []
    for i in resultList:
        ids.append(data.loc[i,'.I'].to_list()[0])
        # print(data.loc[i,'.I'].to_list()[0])
    return ids
    

In [ ]:
search(test)

### correction

In [ ]:
import nltk
from nltk.metrics.distance import edit_distance
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from nltk.corpus import words

In [ ]:
correct_words = words.words()
incorrectWords = '''naem'''.split()
for word in incorrectWords:
    try:
        temp = [(jaccard_distance(set(ngrams(word, 2)),
                                  set(ngrams(w, 2))),w)
                                  for w in correct_words if w[0] == word[0]]
        print(sorted(temp, key = lambda val:val[0])[0][1])
    except:
        pass

In [ ]:
correct_words = words.words()
incorrectWords = '''helllo my name iss saher'''.split()
for word in incorrectWords:
    try:
        temp = [(edit_distance(word,w),w)
                                  for w in correct_words if w[0] == word[0]]
        print(sorted(temp, key = lambda val:val[0])[0][1])
    except:
        pass

### querying

In [5]:
queriesPath = '../../cacmData/cacmQueryPreprocessed.csv'
queriesData = pd.read_csv(queriesPath)
queriesData.head()

,.I,data
0,1,articl exist deal tss time share system oper s...
1,2,interest articl write either priev udo pooch p...
2,3,intermedi languag use construct multi target c...
3,4,interest mechan commun disjoint process possib...
4,5,like paper design implement edit interfac wind...


In [6]:

def queryingData(qDataFrame:pd.DataFrame, n):
    result = pd.DataFrame()
    resultDict:dict = {}
    resultDictCopy = resultDict.copy()
    for i in qDataFrame.index:
        try:
            tempList:list = search(qDataFrame.loc[i,'data'], n)
            for id in range(1,n+1):
                resultDictCopy[str(id)] = tempList[id - 1]
            result = result.append(resultDictCopy, ignore_index=True)
            resultDictCopy = resultDict.copy()
        except:
            print(i)
            raise
    return result





In [ ]:
res = queryingData(queriesData)

In [ ]:
res.head()

In [ ]:
res.to_csv('../../cacmData/cacmQueryResult.csv')

In [8]:
qresPath = '../../cacmData/qrels.csv'
qresData = pd.read_csv(qresPath)
qresData.head()

,ids,data
0,1,1410
1,1,1572
2,1,1605
3,1,2020
4,1,2358


### Evaluate

precision

In [65]:
import pandas as pd
from sklearn.metrics import precision_score

def reSizeLists(l1:list, l2:list):
    if len(l1) < len(l2):
        l2 = l2[0:len(l1)]
    while len(l1) > len(l2):
        l1 = l1[0:len(l2)]

    return l1, l2, len(l1)


def calcPrecisionAtK(resData:pd.DataFrame, qresData: pd.DataFrame):
    precisionsAtK:list = []
    precisionAtK:float
    for i in resData.index:
        precisionOnQuery = []
        resArray = resData.loc[i].to_numpy()
        qresArray = qresData.loc[qresData['ids'] == i+1, 'data'].to_numpy()
        resArray, qresArray, lenth = reSizeLists(resArray, qresArray)
        for lenI in range(0,lenth):
            tempRes:list = resArray[0:lenI+1].tolist()
            tempQRes:list = qresArray[0:lenI+1].tolist()
            precisionOnQuery.append(precision_score(tempQRes, tempRes, average='micro', zero_division=1))
            if i == 1:
                print(tempRes)
                print(tempQRes)
                print(precisionOnQuery)

        precisionsAtK.append(sum(precisionOnQuery) / (lenth + 1))
    precisionAtK = sum(precisionsAtK) / len(precisionsAtK)
    return precisionAtK


    



In [66]:


def getMaxNumberOfGoldenStandard(queriesData):
    n:int = 0
    for i in queriesData.index:
        tempN = len(qresData.loc[qresData['ids'] == i+1, 'data'].to_numpy())
        if n < tempN:
            n = tempN
    return n

def calcCacmDataPrecision(queriesData, qresData):
    n:int = getMaxNumberOfGoldenStandard(queriesData)
    resArray = queryingData(queriesData, n)
    precision = calcPrecisionAtK(resArray, qresData)
    return precision
resArray = calcCacmDataPrecision(queriesData, qresData)
resArray




[3078.0]
[2434]
[0.0]
[3078.0, 236.0]
[2434, 2863]
[0.0, 0.0]
[3078.0, 236.0, 1183.0]
[2434, 2863, 3078]
[0.0, 0.0, 0.0]


0.042414190584943015